In [18]:
import requests

url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchDestination"

querystring = {"query":"new york"}

headers = {
	"x-rapidapi-key": "369ec9cb7fmsh9b4d06e195e671fp1b8c54jsn5cc22233d54b",
	"x-rapidapi-host": "booking-com15.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'status': True, 'message': 'Success', 'timestamp': 1763617139634, 'data': [{'dest_id': '20088325', 'search_type': 'city', 'country': 'United States', 'hotels': 2958, 'type': 'ci', 'region': 'New York', 'label': 'New York, New York, United States', 'dest_type': 'city', 'image_url': 'https://cf.bstatic.com/xdata/images/city/150x150/977436.jpg?k=ee0baaf82d482fae02bcb8dda25be1abff1da43d187bbb26ae6b2f3c9b5f4e61&o=', 'lc': 'en', 'city_ufi': None, 'nr_hotels': 2958, 'cc1': 'us', 'name': 'New York', 'latitude': 40.768074, 'roundtrip': 'GhA0ZTNmMjdiOTNlZmIwMWZlIAAoATICZW46CG5ldyB5b3JrQABKAFAA', 'longitude': -73.981895, 'city_name': 'New York'}, {'dest_id': '929', 'search_type': 'district', 'nr_hotels': 1488, 'cc1': 'us', 'city_ufi': 20088325, 'lc': 'en', 'image_url': 'https://cf.bstatic.com/xdata/images/district/150x150/57691.jpg?k=de3390bd7f9a501b600f4954a39cb97aefe8ea5acaf438d75d689219a94fe981&o=', 'dest_type': 'district', 'label': 'Manhattan, New York, New York, United States', 'region': 'N

In [19]:
response.json()['data'][0]

{'dest_id': '20088325',
 'search_type': 'city',
 'country': 'United States',
 'hotels': 2958,
 'type': 'ci',
 'region': 'New York',
 'label': 'New York, New York, United States',
 'dest_type': 'city',
 'image_url': 'https://cf.bstatic.com/xdata/images/city/150x150/977436.jpg?k=ee0baaf82d482fae02bcb8dda25be1abff1da43d187bbb26ae6b2f3c9b5f4e61&o=',
 'lc': 'en',
 'city_ufi': None,
 'nr_hotels': 2958,
 'cc1': 'us',
 'name': 'New York',
 'latitude': 40.768074,
 'roundtrip': 'GhA0ZTNmMjdiOTNlZmIwMWZlIAAoATICZW46CG5ldyB5b3JrQABKAFAA',
 'longitude': -73.981895,
 'city_name': 'New York'}

In [ ]:
keys =  { "rapidapi-key": "369ec9cb7fmsh9b4d06e195e671fp1b8c54jsn5cc22233d54b", "rapidapi-host": "booking-com15.p.rapidapi.com" }

class BookingRequirements:
    def __init__(self, check_in: str, check_out: str, n_rooms: int, n_adults : int ,  country: str, children_ages: list,  
                 min_price: int = None, max_price: int = None, currency : str = "USD"):
        self.check_in = check_in
        self.check_out = check_out
        self.n_rooms = n_rooms
        self.n_adults = n_adults
        self.country = country
        self.children_ages = children_ages
        self.min_price = min_price
        self.max_price = max_price
        self.currency = currency
    def __repr__(self):
        return f"BookingRequirements(check_in={self.check_in}, check_out={self.check_out}, n_rooms={self.n_rooms}, n_adults={self.n_adults}, country={self.country}, children_ages={self.children_ages}, min_price={self.min_price}, max_price={self.max_price}, currency={self.currency})"
    def to_dict(self):
        return {
            "check_in": self.check_in,
            "check_out": self.check_out,
            "n_rooms": self.n_rooms,
            "n_adults": self.n_adults,
            "country": self.country,
            "children_ages": self.children_ages,
            "min_price": self.min_price,
            "max_price": self.max_price,
            "currency": self.currency
        }

class BookingQuery:
    def __init__(self, query: str, target_loc: str, search_type: str, requirements: BookingRequirements):
        self.query = query
        self.target_loc = target_loc
        self.search_type = search_type
        self.requirements = requirements
    def __repr__(self):
        return f"BookingQuery(query={self.query}, target_loc={self.target_loc}, search_type={self.search_type}, requirements={self.requirements})"
reqs = BookingRequirements("2024-12-01", "2024-12-05", 1, 2, "JP", [], currency ="USD")
booking = BookingQuery("shinjuku", "shinjuku", "district", reqs)

In [ ]:

class BookingCode:
    def __init__(self, response: dict):
        self.name = response.get('name')
        self.search_type = response.get('search_type')
        self.location = response.get('cc1', None)
    def __repr__(self):
        return f"BookingCode(name='{self.name}', search_type='{self.search_type}')"
class Location:

    def __init__(self, query: BookingQuery, keys: dict):
        self.query = query
        self.keys = keys
        avail = self.get_available_locations()
        self.code = self.get_city_code(avail, query.target_loc), query.search_type
        self.requirements = query.requirements


    def get_available_locations(self) -> dict:
        url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchDestination"
        querystring = {"query": self.query.query}
        headers = {
            "x-rapidapi-key": self.keys['rapidapi-key'],
            "x-rapidapi-host": self.keys['rapidapi-host']
        }
        response = requests.get(url, headers=headers, params=querystring)
        return response.json()
    
    def get_city_code(self, response : dict, target_loc: str, search_type : str = 'district') -> str:
        search_types = ['city', 'region', 'landmark', 'district']
        if search_type not in search_types:
            raise ValueError(f"search_type must be one of {search_types}")
        for name in response.get('data'):
            if  target_loc.lower() in name.get('name').lower().split(" ") and name.get('search_type') == search_type:
                print("Found city:", name.get('name'))
                return BookingCode(name)
        
        for type in search_types:
            for name in response.get('data'):
                if  target_loc.lower() in name.get('name').lower().split(" ") and name.get('search_type') == type:
                    print(f"Found location with different search type ({type}):", name.get('name'))
                    return BookingCode(name)
                
    def get_hotels(self, booking : BookingCode) -> dict:
        url = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels"
        requirements = self.requirements

        querystring = {"dest_id":booking.name ,
                    "search_type":booking.search_type,
                    "adults":requirements.n_adults,
                    "checkin_date":requirements.check_in,
                    "checkout_date":requirements.check_out,
                    "room_qty": requirements.n_rooms,
                    "children_age": str(requirements.children_ages).replace(" ", "").replace("[", "").replace("]", ""),
                    "languagecode":"en-us","currency_code":requirements.currency, "location": booking.location}

        headers = {
            "x-rapidapi-key": "369ec9cb7fmsh9b4d06e195e671fp1b8c54jsn5cc22233d54b",
            "x-rapidapi-host": "booking-com15.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)

        print(response.json())
    def __repr__(self):
        return f"Location(code={self.code}, query={self.query})"

Location(booking, keys)

# get_city_code(response.json(), "shinjuku", 'region')

Found city: Shinjuku Area


Location(code=(BookingCode(name='Shinjuku Area', search_type='district'), 'district'), query=BookingQuery(query=shinjuku, target_loc=shinjuku, search_type=district, requirements=BookingRequirements(check_in=2024-12-01, check_out=2024-12-05, n_rooms=1, n_adults=2, country=JP, children_ages=[], min_price=None, max_price=None, currency=USD)))

[{'dest_id': '3054',
  'search_type': 'district',
  'city_name': 'Tokyo',
  'roundtrip': 'GhA0M2Y2NDE4MTA3OGUwMDFjIAAoATICZW46CHNoaW5qdWt1QABKAFAA',
  'city_ufi': -246227,
  'country': 'Japan',
  'type': 'di',
  'image_url': 'https://cf.bstatic.com/xdata/images/district/150x150/38059.jpg?k=070b908d2aefcecfb05f99b1af1f133cfbfe2667c98e1c1177ac0369ae74b84c&o=',
  'lc': 'en',
  'cc1': 'jp',
  'latitude': 35.690914,
  'nr_hotels': 1070,
  'dest_type': 'district',
  'longitude': 139.7045,
  'label': 'Shinjuku Area, Tokyo, Tokyo-to, Japan',
  'hotels': 1070,
  'name': 'Shinjuku Area',
  'region': 'Tokyo-to'},
 {'dest_id': '257062',
  'search_type': 'landmark',
  'longitude': 139.70026,
  'label': 'Shinjuku Station, Tokyo, Tokyo-to, Japan',
  'dest_type': 'landmark',
  'nr_hotels': 1045,
  'region': 'Tokyo-to',
  'name': 'Shinjuku Station',
  'hotels': 1045,
  'landmark_type': 2,
  'type': 'la',
  'city_ufi': -246227,
  'country': 'Japan',
  'roundtrip': 'GhA0M2Y2NDE4MTA3OGUwMDFjIAEoATICZW46CH

'1, 2, 3'